In [248]:
import random
import warnings

warnings.filterwarnings("ignore")
random.seed(1234)

In [249]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from utilities.data_loader import load_data
from utilities.data_formater import format_data
from models.mlp_model_torch import NeuralNetwork
from training_testing.regularization_testing import train_and_evaluate_model

torch.manual_seed(1234)

In [250]:
# Load and prepare your dataset
X_train, y_train = load_data("data/center_surround_train.csv")
X_valid, y_valid = load_data("data/center_surround_valid.csv")
X_test, y_test = load_data("data/center_surround_test.csv")

# Reshape targets if needed
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [251]:
# Prepare datasets
train_dataset = format_data(X_train, y_train)
valid_dataset = format_data(X_valid, y_valid)
test_dataset = format_data(X_test, y_test)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [252]:
# Convert your data into PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_valid_tensor = torch.FloatTensor(X_valid)
y_valid_tensor = torch.FloatTensor(y_valid)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Wrap them in DataLoader
batch_size = 64  # Adjust as needed
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [253]:
input_size = X_train.shape[1]
hidden_size = 15
output_size = 1
num_epochs = 400

print("Training with L2 Regularization (Weight Decay)")
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
train_and_evaluate_model(model, criterion, optimizer, train_loader, valid_loader, test_loader, num_epochs)


# L1 Regularization
print("\nTraining with L1 Regularization")
model = NeuralNetwork(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # No weight_decay here for L1
train_and_evaluate_model(model, criterion, optimizer, train_loader, valid_loader, test_loader, num_epochs, l1_strength=1e-5)


# Dropout
print("\nTraining with Dropout")
model = NeuralNetwork(input_size, hidden_size, output_size, dropout_rate=0.5)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # No weight_decay necessary for Dropout
train_and_evaluate_model(model, criterion, optimizer, train_loader, valid_loader, test_loader, num_epochs)


Training with L2 Regularization (Weight Decay)
Train Loss: 0.0756, Train Accuracy: 30.8800
Valid Loss: 0.1036, Valid Accuracy: 31.1600
Test Loss: 0.1901, Test Accuracy: 30.0400

Training with L1 Regularization
Train Loss: 0.0784, Train Accuracy: 30.8800
Valid Loss: 0.1111, Valid Accuracy: 31.1600
Test Loss: 0.1825, Test Accuracy: 30.0400

Training with Dropout
Train Loss: 0.1066, Train Accuracy: 31.1600
Valid Loss: 0.0657, Valid Accuracy: 31.1600
Test Loss: 0.1867, Test Accuracy: 30.0400
